# Titanic Survival

#### The challenge:

Build a predictive model that answers the question: "what sorts of people were more likely to survive the Titanic sinking?

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [12]:
df = pd.concat([test, train])
df.head(-10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,NaN
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,NaN
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,NaN
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,NaN
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
876,877,3,"Gustafsson, Mr. Alfred Ossian",male,20.0,0,0,7534,9.8458,NaN,S,0.0
877,878,3,"Petroff, Mr. Nedelio",male,19.0,0,0,349212,7.8958,NaN,S,0.0
878,879,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S,0.0
879,880,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C,1.0


In [10]:
df.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Survived
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,891.000000
mean,655.000000,2.294882,29.881138,0.498854,0.385027,33.295479,0.383838
std,378.020061,0.837836,14.413493,1.041658,0.865560,51.758668,0.486592
min,1.000000,1.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,328.000000,2.000000,21.000000,0.000000,0.000000,7.895800,0.000000
50%,655.000000,3.000000,28.000000,0.000000,0.000000,14.454200,0.000000
75%,982.000000,3.000000,39.000000,1.000000,0.000000,31.275000,1.000000
max,1309.000000,3.000000,80.000000,8.000000,9.000000,512.329200,1.000000


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Name         1309 non-null   object 
 3   Sex          1309 non-null   object 
 4   Age          1046 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Ticket       1309 non-null   object 
 8   Fare         1308 non-null   float64
 9   Cabin        295 non-null    object 
 10  Embarked     1307 non-null   object 
 11  Survived     891 non-null    float64
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


Preguntas técnicas.  
¿Qué se hace cuando un pasajero no tiene una edad registrada (campo nulo)?    
¿Cómo se reemplazan los datos numéricos y categóricos nulos?

### Descripción de los datos

- **PassengerId**  
    - Id del pasajero. 
    - ¿Se elimina dicha variable? No se debería de tener en cuenta para el modelo.
- **Pclass**  
    - Clase del Tiquete
    - 1 = Primera clase, 2 = Segunda clase, 3 = Tercera clase
- **Name**:  
    - Nombre del pasajero
    - No se tiene en cuenta para el modelo
- **Sex**:  
    - Sexo del pasajero. male o female.
- **Age**:  
    - Edad del pasajero.
- **SibSp**:  
    - Número de hermanos / Cónyuges a bordo del Titanic.
- **Parch**:  
    - Número de padres / Niños a bordo del Titanic.
- **Ticket**:  
    - Número del tiquete
- **Fare**:  
    - Tarifa del pasajero
- **Cabin**: 
    - Número de cabina
- **Embarked**:  
    - Puerto de embarcación
    - C = Cherbourg, Q = Queenstown, S = Southampton  
- **Survided**:
    - Variable objetivo
    - ¿Un pasajero sobrevivió? 0 = No,  1 = Si
    - 891 datos no nulos. El resto se debería eliminar (?

In [ ]:
**Variables que se eliminarían por irrelevantes**:
    - PassengerId
    - Name
    - Fare